In [1]:
import os
import pandas as pd
import numpy as np
import csv
import json
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf


from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline



In [2]:
# The main dataset
imdb_filepath = os.path.join(os.getcwd(), 'datasets', 'IMDB-Movie-Data.csv')
imdb_raw = pd.read_csv(imdb_filepath)

#The best picture dataset
oscardf1_filepath = os.path.join(os.getcwd(), 'datasets', 'oscars_df.csv')
oscardf1 = pd.read_csv(oscardf1_filepath)

# the Oscar Award dataset
oscardf2_filepath = os.path.join(os.getcwd(), 'datasets', 'oscars_df2.csv')
oscardf2 = pd.read_csv(oscardf2_filepath)

# the Tmdb dataset
tmdb10000_filepath = os.path.join(os.getcwd(), 'datasets', 'TMDB_10000_Popular_Movies.csv')
tmdb10000 = pd.read_csv(tmdb10000_filepath)

# tmdbClean_filepath = os.path.join(os.getcwd(), 'datasets', 'AllMoviesDetailsCleaned.csv')
# tmdbClean = pd.read_csv(tmdbClean_filepath,on_bad_lines='skip')



## There are many columns we created, but the only columns we used for study are described in our PDF

In [3]:
oscardf2

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
...,...,...,...,...,...,...,...
10760,2022,2023,95,HONORARY AWARD,"To Euzhan Palcy, a masterful filmmaker who bro...",NaN,True
10761,2022,2023,95,HONORARY AWARD,"To Diane Warren, for her genius, generosity an...",NaN,True
10762,2022,2023,95,HONORARY AWARD,"To Peter Weir, a fearless and consummate filmm...",NaN,True
10763,2022,2023,95,GORDON E. SAWYER AWARD,Iain Neil,NaN,True


In [4]:
# oscar2['year_film'] = pd.to_datetime(oscar2['year_film'])

oscardf2 = oscardf2[~(oscardf2['year_film'] >= 2016)]
oscardf2 = oscardf2[~(oscardf2['year_film'] <= 1940)]
oscardf2

,year_film,year_ceremony,ceremony,category,name,film,winner
1087,1941,1942,14,ACTOR,Gary Cooper,Sergeant York,True
1088,1941,1942,14,ACTOR,Cary Grant,Penny Serenade,False
1089,1941,1942,14,ACTOR,Walter Huston,All That Money Can Buy,False
1090,1941,1942,14,ACTOR,Robert Montgomery,Here Comes Mr. Jordan,False
1091,1941,1942,14,ACTOR,Orson Welles,Citizen Kane,False
...,...,...,...,...,...,...,...
9884,2015,2016,88,WRITING (Original Screenplay),Written by Josh Singer & Tom McCarthy,Spotlight,True
9885,2015,2016,88,WRITING (Original Screenplay),Screenplay by Jonathan Herman and Andrea Berlo...,Straight Outta Compton,False
9886,2015,2016,88,JEAN HERSHOLT HUMANITARIAN AWARD,Debbie Reynolds,NaN,True
9887,2015,2016,88,HONORARY AWARD,Spike Lee,NaN,True


In [5]:
# oscardf2 = oscardf2[((oscardf2['category'] == 'ACTOR' )
#                   | (oscardf2['category'] == 'ACTRESS')
#                   | (oscardf2['category'] == 'ACTOR IN A SUPPORTING ROLE')
#                   | (oscardf2['category'] == 'ACTRESS IN A SUPPORTING ROLE')
#                   | (oscardf2['category'] == 'ACTOR IN A LEADING ROLE' )
#                   | (oscardf2['category'] == 'ACTRESS IN A LEADING ROLE')
#                  )
#                ]
# oscardf2
oscarDirector = oscardf2[oscardf2['category'] == 'DIRECTING']

oscardf2 = oscardf2[((oscardf2['category'] == 'ACTOR' )
                  | (oscardf2['category'] == 'ACTRESS')
                  | (oscardf2['category'] == 'ACTOR IN A LEADING ROLE' )
                  | (oscardf2['category'] == 'ACTRESS IN A LEADING ROLE')
                 )
               ]
oscardf2

,year_film,year_ceremony,ceremony,category,name,film,winner
1087,1941,1942,14,ACTOR,Gary Cooper,Sergeant York,True
1088,1941,1942,14,ACTOR,Cary Grant,Penny Serenade,False
1089,1941,1942,14,ACTOR,Walter Huston,All That Money Can Buy,False
1090,1941,1942,14,ACTOR,Robert Montgomery,Here Comes Mr. Jordan,False
1091,1941,1942,14,ACTOR,Orson Welles,Citizen Kane,False
...,...,...,...,...,...,...,...
9775,2015,2016,88,ACTRESS IN A LEADING ROLE,Cate Blanchett,Carol,False
9776,2015,2016,88,ACTRESS IN A LEADING ROLE,Brie Larson,Room,True
9777,2015,2016,88,ACTRESS IN A LEADING ROLE,Jennifer Lawrence,Joy,False
9778,2015,2016,88,ACTRESS IN A LEADING ROLE,Charlotte Rampling,45 Years,False


In [6]:
#=============movie director
# Notice that though the warning says "A value is trying to be set on a copy of a slice from a DataFrame", 
#the original dataset still got modified
imdb_raw['OscarDirectorNomineeWinner'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    for j in range(0, oscarDirector.shape[0]):
        if (imdb_raw['Director'].iloc[i]==oscarDirector['name'].iloc[j] 
           and oscarDirector['year_film'].iloc[j] < imdb_raw['Year'].iloc[i]):
            
            imdb_raw['OscarDirectorNomineeWinner'].iloc[i]=1
            break

/tmp/ipykernel_88/1616866839.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['OscarDirectorNomineeWinner'].iloc[i]=1


In [7]:
imdb_raw['OscarDirectorWinner'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    for j in range(0, oscarDirector.shape[0]):
        if (imdb_raw['Director'].iloc[i]==oscarDirector['name'].iloc[j] 
           and oscarDirector['year_film'].iloc[j] < imdb_raw['Year'].iloc[i] 
           and oscarDirector['winner'].iloc[j] ==True ):
            imdb_raw['OscarDirectorWinner'].iloc[i]=1
            break

/tmp/ipykernel_88/484001532.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['OscarDirectorWinner'].iloc[i]=1


In [8]:
imdb_raw['OscarDirectorMix'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    for j in range(0, oscarDirector.shape[0]):

        if (imdb_raw['Director'].iloc[i]==oscarDirector['name'].iloc[j] 
           and oscarDirector['year_film'].iloc[j] < imdb_raw['Year'].iloc[i] 
           and oscarDirector['winner'].iloc[j] ==True ):
            imdb_raw['OscarDirectorMix'].iloc[i]=2
            break
        elif(imdb_raw['Director'].iloc[i]==oscarDirector['name'].iloc[j] 
           and oscarDirector['year_film'].iloc[j] < imdb_raw['Year'].iloc[i] 
           and oscarDirector['winner'].iloc[j] ==False ):
            imdb_raw['OscarDirectorMix'].iloc[i]=1

#=================================================

/tmp/ipykernel_88/3163689423.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['OscarDirectorMix'].iloc[i]=1
/tmp/ipykernel_88/3163689423.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['OscarDirectorMix'].iloc[i]=2


In [9]:
imdb_raw.fillna({'OscarDirectorNomineeWinner':0, 'OscarDirectorWinner':0,  'OscarDirectorMix':0,}, inplace=True)
imdb_raw

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,OscarDirectorNomineeWinner,OscarDirectorWinner,OscarDirectorMix
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,0.0,0.0,0.0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,1.0,0.0,1.0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,1.0,0.0,1.0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,0.0,0.0,0.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,0.0,0.0,0.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,0.0,0.0,0.0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,0.0,0.0,0.0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0,0.0,0.0,0.0


In [10]:
imdb_raw['hasOscarActorNomineeWinner'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    
    #imdbmerge2['Actors'.]
    list = imdb_raw['Actors'].iloc[i].split(',')

    for j in range(0, len(list)):
        if  imdb_raw['hasOscarActorNomineeWinner'].iloc[i]==1:
            break
        for k in range (oscardf2.shape[0]):
            if(list[j]==oscardf2['name'].iloc[k] 
               and oscardf2['year_film'].iloc[k] < imdb_raw['Year'].iloc[i]):
                imdb_raw['hasOscarActorNomineeWinner'].iloc[i]=1
                break
    

/tmp/ipykernel_88/3891524838.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['hasOscarActorNomineeWinner'].iloc[i]=1


In [11]:
imdb_raw['hasOscarActorWinner'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    
    #imdbmerge2['Actors'.]
    list = imdb_raw['Actors'].iloc[i].split(',')

    for j in range(0, len(list)):
        if  imdb_raw['hasOscarActorWinner'].iloc[i]==1:
            break
        for k in range (oscardf2.shape[0]):
            if(list[j]==oscardf2['name'].iloc[k] 
               and oscardf2['year_film'].iloc[k] < imdb_raw['Year'].iloc[i] 
               and oscardf2['winner'].iloc[k] == True):
                imdb_raw['hasOscarActorWinner'].iloc[i]=1
                break

/tmp/ipykernel_88/1131206366.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['hasOscarActorWinner'].iloc[i]=1


In [12]:
imdb_raw['hasOscarActorMix'] = np.nan
for i in range(0, imdb_raw.shape[0]):
    
    #imdbmerge2['Actors'.]
    list = imdb_raw['Actors'].iloc[i].split(',')

    for j in range(0, len(list)):
        if  imdb_raw['hasOscarActorMix'].iloc[i] == 2:
            break
        for k in range (oscardf2.shape[0]):
            if(list[j]==oscardf2['name'].iloc[k] 
               and oscardf2['year_film'].iloc[k] < imdb_raw['Year'].iloc[i] 
               and oscardf2['winner'].iloc[k] == True):
                imdb_raw['hasOscarActorMix'].iloc[i]=2
                break
            elif (list[j]==oscardf2['name'].iloc[k] 
               and oscardf2['year_film'].iloc[k] < imdb_raw['Year'].iloc[i] 
               and oscardf2['winner'].iloc[k] == False):
                imdb_raw['hasOscarActorMix'].iloc[i]=1
                
                break

/tmp/ipykernel_88/20946579.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['hasOscarActorMix'].iloc[i]=2
/tmp/ipykernel_88/20946579.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imdb_raw['hasOscarActorMix'].iloc[i]=1


In [13]:
imdb_raw

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,OscarDirectorNomineeWinner,OscarDirectorWinner,OscarDirectorMix,hasOscarActorNomineeWinner,hasOscarActorWinner,hasOscarActorMix
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,0.0,0.0,0.0,NaN,NaN,NaN
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,1.0,0.0,1.0,NaN,NaN,NaN
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,1.0,0.0,1.0,NaN,NaN,NaN
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,0.0,0.0,0.0,1.0,1.0,2.0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,0.0,0.0,0.0,1.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,0.0,0.0,0.0,1.0,NaN,1.0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,0.0,0.0,0.0,NaN,NaN,NaN
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,0.0,0.0,0.0,NaN,NaN,NaN
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0,0.0,0.0,0.0,NaN,NaN,NaN


In [14]:
# imdb_raw['OscarBestPictureNominee'] = np.nan
oscardf1 = oscardf1[['Year of Release', 'Film', 'Award']]

In [15]:
oscardf1

,Year of Release,Film,Award
0,1927,Wings,Winner
1,1927,7th Heaven,Nominee
2,1928,The Racket,Nominee
3,1929,The Broadway Melody,Winner
4,1929,Alibi,Nominee
...,...,...,...
566,2020,Mank,Nominee
567,2020,Minari,Nominee
568,2020,Promising Young Woman,Nominee
569,2019,Sound of Metal,Nominee


In [16]:
imdb_raw = pd.merge(imdb_raw, oscardf1, left_on=['Title', 'Year'],right_on=['Film', 'Year of Release'], how='left')
#imdb_raw = pd.merge(imdb_raw, oscardf1, left_on=['Title', 'Year'],right_on=['Film', ''], how='left')
imdb_raw.fillna({'Award':0}, inplace=True)
imdb_raw

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,Metascore,OscarDirectorNomineeWinner,OscarDirectorWinner,OscarDirectorMix,hasOscarActorNomineeWinner,hasOscarActorWinner,hasOscarActorMix,Year of Release,Film,Award
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,76.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,65.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,62.0,1.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,59.0,0.0,0.0,0.0,1.0,1.0,2.0,NaN,NaN,0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,40.0,0.0,0.0,0.0,1.0,NaN,1.0,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,45.0,0.0,0.0,0.0,1.0,NaN,1.0,NaN,NaN,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,46.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,50.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,22.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0


In [17]:
imdb_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 0 to 999
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Rank                        1000 non-null   int64  
 1   Title                       1000 non-null   object 
 2   Genre                       1000 non-null   object 
 3   Description                 1000 non-null   object 
 4   Director                    1000 non-null   object 
 5   Actors                      1000 non-null   object 
 6   Year                        1000 non-null   int64  
 7   Runtime (Minutes)           1000 non-null   int64  
 8   Rating                      1000 non-null   float64
 9   Votes                       1000 non-null   int64  
 10  Revenue (Millions)          872 non-null    float64
 11  Metascore                   936 non-null    float64
 12  OscarDirectorNomineeWinner  1000 non-null   float64
 13  OscarDirectorWinner         1000 n

In [18]:
imdb_raw.drop(labels=['Film', 'Year of Release'], axis=1, inplace=True)

In [19]:
imdb_raw.fillna({'hasOscarActorNomineeWinner':0, 'hasOscarActorWinner':0,  'hasOscarActorMix':0,}, inplace=True)
imdb_raw

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,Revenue (Millions),Metascore,OscarDirectorNomineeWinner,OscarDirectorWinner,OscarDirectorMix,hasOscarActorNomineeWinner,hasOscarActorWinner,hasOscarActorMix,Award
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,333.13,76.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,126.46,65.0,1.0,0.0,1.0,0.0,0.0,0.0,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,138.12,62.0,1.0,0.0,1.0,0.0,0.0,0.0,0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,270.32,59.0,0.0,0.0,0.0,1.0,1.0,2.0,0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,325.02,40.0,0.0,0.0,0.0,1.0,0.0,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,NaN,45.0,0.0,0.0,0.0,1.0,0.0,1.0,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,17.54,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,58.01,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,NaN,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [20]:
#===================
imdb_raw['OscarPictureWinner']= imdb_raw['Award']
imdb_raw['OscarPictureWinner']=  imdb_raw['OscarPictureWinner'].apply(lambda x: 1 if x == 'Winner' else 0)
#======a===============
imdb_raw['OscarPictureNomineeWinner'] = imdb_raw['Award']
imdb_raw['OscarPictureNomineeWinner']= imdb_raw['OscarPictureNomineeWinner'].apply(lambda x: 1 if x != 0 else 0)
#=============
imdb_raw['OscarPictureMix'] = imdb_raw['Award']
imdb_raw['OscarPictureMix'] = imdb_raw['OscarPictureMix'].apply(lambda x: 2 if x == 'Winner'else x)
imdb_raw['OscarPictureMix'] = imdb_raw['OscarPictureMix'].apply(lambda x: 1 if x == 'Nominee' else x)

imdb_raw.drop(labels=['Award'], axis=1, inplace=True)



In [21]:
genreList = imdb_raw['Genre'].str.get_dummies(sep=",")
imdb_raw=pd.concat([imdb_raw, genreList], axis=1)
imdb_raw
#imdb_raw.drop(['Genre'],axis=1, inplace=True)

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,0,0,0,1,0,0,0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,0,0,1,0,1,0,0,0,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,1,0,0,0,0,0,0,1,0,0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,0,0,0,0,0,0,0,0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,0,0,1,0,0,0,0,0,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,1,0,0,0,0,0,0,0,0,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,1,0,0,1,0,0,0,0,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,0,0,0,0,0,0,0,0


In [22]:
print(imdb_raw['hasOscarActorWinner'].sum())
print(imdb_raw['hasOscarActorNomineeWinner'].sum())
print(imdb_raw['hasOscarActorMix'].sum())
print(imdb_raw['OscarPictureNomineeWinner'].sum())
print(imdb_raw['OscarPictureWinner'].sum())
print(imdb_raw['OscarPictureMix'].sum())
print('director', imdb_raw['OscarDirectorNomineeWinner'].sum())
print(imdb_raw['OscarDirectorWinner'].sum())
print(imdb_raw['OscarDirectorMix'].sum())


91.0
259.0
300.0
69
10
79
director 115.0
50.0
165.0


In [23]:
imdb_raw[(imdb_raw['OscarPictureMix'] == 2)]
imdb_raw[(imdb_raw['hasOscarActorMix'] == 2)]
imdb_raw[(imdb_raw['hasOscarActorMix'] == 1)]


,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,0,0,0,0,0,0,0,0
5,6,The Great Wall,"Action,Adventure,Fantasy",European mercenaries searching for black powde...,Yimou Zhang,"Matt Damon, Tian Jing, Willem Dafoe, Andy Lau",2016,103,6.1,56036,...,0,0,0,0,0,0,0,0,0,0
6,7,La La Land,"Comedy,Drama,Music",A jazz pianist falls for an aspiring actress i...,Damien Chazelle,"Ryan Gosling, Emma Stone, Rosemarie DeWitt, J....",2016,128,8.3,258682,...,0,1,0,0,0,0,0,0,0,0
7,8,Mindhorn,Comedy,A has-been actor best known for playing the ti...,Sean Foley,"Essie Davis, Andrea Riseborough, Julian Barrat...",2016,89,6.4,2490,...,0,0,0,0,0,0,0,0,0,0
9,10,Passengers,"Adventure,Drama,Romance",A spacecraft traveling to a distant colony pla...,Morten Tyldum,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",2016,116,7.0,192177,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
964,965,Custody,Drama,The lives of three women are unexpectedly chan...,James Lapine,"Viola Davis, Hayden Panettiere, Catalina Sandi...",2016,104,6.9,280,...,0,0,0,0,0,0,0,0,0,0
965,966,Inland Empire,"Drama,Mystery,Thriller",As an actress starts to adopt the persona of h...,David Lynch,"Laura Dern, Jeremy Irons, Justin Theroux, Karo...",2006,180,7.0,44227,...,0,0,0,1,0,0,0,1,0,0
969,970,The Lone Ranger,"Action,Adventure,Western",Native American warrior Tonto recounts the unt...,Gore Verbinski,"Johnny Depp, Armie Hammer, William Fichtner,To...",2013,150,6.5,190855,...,0,0,0,0,0,0,0,0,0,1
981,982,Annie,"Comedy,Drama,Family","A foster kid, who lives with her mean foster m...",Will Gluck,"Quvenzhané Wallis, Cameron Diaz, Jamie Foxx, R...",2014,118,5.3,27312,...,0,0,0,0,0,0,0,0,0,0


### Clean each dataset:tmdb10000

In [24]:
tmdb10000 = tmdb10000[['Title', 'Release_Date','Budget','Original_Language']]
tmdb10000['Release_Date']= pd.to_datetime(tmdb10000['Release_Date'],format='%Y-%m-%d')
tmdb10000['year']= tmdb10000['Release_Date'].dt.year
tmdb10000 = tmdb10000[~tmdb10000.duplicated(keep='first')]
tmdb10000 =tmdb10000.drop(['Release_Date'], axis=1)
print(tmdb10000.duplicated().sum())
tmdb10000


1


,Title,Budget,Original_Language,year
0,Ad Astra,87500000.0,en,2019.0
1,Bloodshot,42000000.0,en,2020.0
2,Bad Boys for Life,0.0,en,2020.0
3,Ant-Man,130000000.0,en,2015.0
4,Percy Jackson: Sea of Monsters,90000000.0,en,2013.0
...,...,...,...,...
9995,Bliss Point,0.0,nl,2017.0
9996,The Reluctant Dragon,600.0,en,1941.0
9997,Dead Birds,1500000.0,en,2005.0
9998,End Game,5000000.0,en,2006.0


### merge tmdb10000

In [25]:
imdbmerge1 = pd.merge(imdb_raw, tmdb10000, left_on=['Title','Year'],right_on=['Title', 'year'], how='left')
imdbmerge1['Budget'] = imdbmerge1['Budget'].replace(0, np.nan)
imdbmerge1.drop(['year'], axis=1, inplace=True )
imdbmerge1

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western,Budget,Original_Language
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,0,1,0,0,0,0,170000000.0,en
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,1,0,1,0,0,0,0,130000000.0,en
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,0,0,0,0,0,1,0,0,9000000.0,en
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,0,0,0,0,0,0,75000000.0,en
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,0,0,0,0,0,0,176000000.0,en
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,1,0,0,0,0,0,0,19500000.0,en
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,0,0,0,0,0,0,0,0,10200000.0,en
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,0,1,0,0,0,0,0,23000000.0,en
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,0,0,0,0,0,0,20000000.0,en


In [26]:
imdbmerge1['Revenue'] = imdbmerge1['Revenue (Millions)']*1000000
imdbmerge1['Budget (Millions)'] = imdbmerge1['Budget']/1000000       
# we find that the movie 'A Kind of Murder''s real revenue is 2915 dollars (domestic box office)
imdbmerge1['Revenue'] = imdbmerge1['Revenue'].replace(0, 2915)
# passengers[passengers['Age'] <= 17]

imdbmerge1['LogRevenue'] = np.log10(imdbmerge1['Revenue'])
imdbmerge1['LogBudget'] = np.log10(imdbmerge1['Budget'])

imdbmerge1['SqrtRevenue'] = np.sqrt(imdbmerge1['Revenue'])
imdbmerge1['SqrtBudget'] = np.sqrt(imdbmerge1['Budget'])
imdbmerge1



,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,War,Western,Budget,Original_Language,Revenue,Budget (Millions),LogRevenue,LogBudget,SqrtRevenue,SqrtBudget
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,170000000.0,en,333130000.0,170.0,8.522614,8.230449,18251.849221,13038.404810
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,0,130000000.0,en,126460000.0,130.0,8.101953,8.113943,11245.443522,11401.754251
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,0,0,9000000.0,en,138120000.0,9.0,8.140257,6.954243,11752.446554,3000.000000
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,75000000.0,en,270320000.0,75.0,8.431878,7.875061,16441.411132,8660.254038
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,176000000.0,en,325020000.0,176.0,8.511910,8.245513,18028.311069,13266.499161
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,0,19500000.0,en,NaN,19.5,NaN,7.290035,NaN,4415.880433
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,0,0,10200000.0,en,17540000.0,10.2,7.244030,7.008600,4188.078318,3193.743885
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,0,23000000.0,en,58010000.0,23.0,7.763503,7.361728,7616.429610,4795.831523
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,20000000.0,en,NaN,20.0,NaN,7.301030,NaN,4472.135955


In [27]:
languageList = imdbmerge1['Original_Language'].str.get_dummies()
imdbmerge1 = pd.concat([imdbmerge1, languageList], axis=1)
imdbmerge1

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,da,de,en,es,fr,hi,it,ko,ru,te
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,1,0,0,0,0,0,0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,0,1,0,0,0,0,0,0,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,0,0,1,0,0,0,0,0,0,0
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,1,0,0,0,0,0,0,0
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,0,1,0,0,0,0,0,0,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,0,0,1,0,0,0,0,0,0,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,0,1,0,0,0,0,0,0,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,1,0,0,0,0,0,0,0


In [28]:
imdbmerge1['Year'] = imdbmerge1['Year'].astype(str)
yearList= imdbmerge1['Year'].str.get_dummies()
imdbmerge1 = pd.concat([imdbmerge1, yearList], axis=1)
imdbmerge1['Year'] = imdbmerge1['Year'].astype(int)

In [29]:
imdbmerge1

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,0,0,0,0,0,1,0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,0,0,0,0,1,0,0,0,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,0,0,0,0,0,0,0,0,0,1
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,0,0,0,0,0,0,0,1
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,0,0,0,0,0,0,0,1,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,1,0,0,0,0,0,0,0,0,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,1,0,0,0,0,0,0,0,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,0,0,0,0,0,1,0,0


In [30]:
imdbmerge1.to_csv('datasets/imdbFinalMerge.csv', index=False)

In [31]:
imdbmerge1['Original_Language'].unique()

array(['en', nan, 'te', 'hi', 'es', 'fr', 'ko', 'de', 'it', 'ru', 'da'],
      dtype=object)

In [32]:
imdbmerge1['Original_Language'].isna().sum()

155

In [33]:
imdbmerge1

,Rank,Title,Genre,Description,Director,Actors,Year,Runtime (Minutes),Rating,Votes,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
0,1,Guardians of the Galaxy,"Action,Adventure,Sci-Fi",A group of intergalactic criminals are forced ...,James Gunn,"Chris Pratt, Vin Diesel, Bradley Cooper, Zoe S...",2014,121,8.1,757074,...,0,0,0,0,0,0,0,1,0,0
1,2,Prometheus,"Adventure,Mystery,Sci-Fi","Following clues to the origin of mankind, a te...",Ridley Scott,"Noomi Rapace, Logan Marshall-Green, Michael Fa...",2012,124,7.0,485820,...,0,0,0,0,0,1,0,0,0,0
2,3,Split,"Horror,Thriller",Three girls are kidnapped by a man with a diag...,M. Night Shyamalan,"James McAvoy, Anya Taylor-Joy, Haley Lu Richar...",2016,117,7.3,157606,...,0,0,0,0,0,0,0,0,0,1
3,4,Sing,"Animation,Comedy,Family","In a city of humanoid animals, a hustling thea...",Christophe Lourdelet,"Matthew McConaughey,Reese Witherspoon, Seth Ma...",2016,108,7.2,60545,...,0,0,0,0,0,0,0,0,0,1
4,5,Suicide Squad,"Action,Adventure,Fantasy",A secret government agency recruits some of th...,David Ayer,"Will Smith, Jared Leto, Margot Robbie, Viola D...",2016,123,6.2,393727,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,Secret in Their Eyes,"Crime,Drama,Mystery","A tight-knit team of rising investigators, alo...",Billy Ray,"Chiwetel Ejiofor, Nicole Kidman, Julia Roberts...",2015,111,6.2,27585,...,0,0,0,0,0,0,0,0,1,0
996,997,Hostel: Part II,Horror,Three American college students studying abroa...,Eli Roth,"Lauren German, Heather Matarazzo, Bijou Philli...",2007,94,5.5,73152,...,1,0,0,0,0,0,0,0,0,0
997,998,Step Up 2: The Streets,"Drama,Music,Romance",Romantic sparks occur between two dance studen...,Jon M. Chu,"Robert Hoffman, Briana Evigan, Cassie Ventura,...",2008,98,6.2,70699,...,0,1,0,0,0,0,0,0,0,0
998,999,Search Party,"Adventure,Comedy",A pair of friends embark on a mission to reuni...,Scot Armstrong,"Adam Pally, T.J. Miller, Thomas Middleditch,Sh...",2014,93,5.6,4881,...,0,0,0,0,0,0,0,1,0,0
